In [1]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import json
import datetime
import functions as f
from tqdm import tqdm
import time

In [2]:
state = pd.read_csv('states.csv')

In [3]:
states = list(state.Abbreviation)

In [4]:
size = '200' #maxsize
engine = db.create_engine('sqlite:///tickets.db')
table = 'events'
frame_list = []
fails = []
success= []


In [5]:
frame_list = []
for state in tqdm(states):
    for page in range(5):
        frame_list.append(f.scrape_ticketmaster(state, size, page))

100%|██████████| 51/51 [03:51<00:00,  3.23s/it]


In [8]:
temp = []
fail = []
for x in frame_list:
    try:
        df = pd.DataFrame(x['_embedded'] ['events'])
        temp.append(df)
    except:
        fail.append(x)

In [10]:
len(temp)

149

In [12]:
fail

[{'_links': {'first': {'href': '/discovery/v2/events.json?countryCode=US&classificationName=music&stateCode=AL&page=0&size=200'},
   'prev': {'href': '/discovery/v2/events.json?countryCode=US&classificationName=music&stateCode=AL&page=1&size=200'},
   'self': {'href': '/discovery/v2/events.json?size=200&countryCode=US&classificationName=music&stateCode=AL&page=2'},
   'last': {'href': '/discovery/v2/events.json?countryCode=US&classificationName=music&stateCode=AL&page=1&size=200'}},
  'page': {'size': 200, 'totalElements': 329, 'totalPages': 2, 'number': 2}},
 {'fault': {'faultstring': 'Spike arrest violation. Allowed rate : MessageRate{messagesPerPeriod=5, periodInMicroseconds=1000000, maxBurstMessageCount=1.0}',
   'detail': {'errorcode': 'policies.ratelimit.SpikeArrestViolation'}}},
 {'_links': {'first': {'href': '/discovery/v2/events.json?countryCode=US&classificationName=music&stateCode=AL&page=0&size=200'},
   'prev': {'href': '/discovery/v2/events.json?countryCode=US&classificat

In [85]:
df = pd.concat(temp)

ValueError: No objects to concatenate

In [86]:
df.reset_index(inplace=True)

In [72]:
df.columns

Index(['index', '_embedded', '_links', 'accessibility', 'address.line1',
       'city.name', 'country.countryCode', 'dates', 'description', 'genre',
       'id', 'images', 'info', 'locale', 'location.latitude',
       'location.longitude', 'max_tickets', 'name_x', 'name_y', 'num_markets',
       'outlets', 'pleaseNote', 'postalCode', 'presales', 'price_max',
       'price_min', 'products', 'promoter', 'promoters', 'public.endDateTime',
       'public.startDateTime', 'seatmap', 'sold_out', 'state.name',
       'state.stateCode', 'subgenre', 'test', 'type', 'url'],
      dtype='object')

In [73]:
df.drop(columns = ['index','_embedded', '_links', 'accessibility','dates','description','images','info', 'locale','outlets','pleaseNote','products',
                   'promoter','promoters','seatmap','test','type','state.name'],inplace=True)

In [75]:
df.rename(columns={'name_x' :'event_name', 'name_y':'venue_name', 'presales': 'is_presale', 'location.latitude':'lat', 'location.longitude':'lon',
                  'address.line1':'address', 'city.name':'city', 'country.countryCode':'county', 'public.endDateTime' : 'event_date',
                  'public.startDateTime':'onsale_date','state.stateCode':'state'},inplace=True)

In [81]:
####fucked up

17       1
18       1
31       1
32       1
33       1
        ..
24160    1
24161    1
24162    1
24164    1
24165    1
Name: sold_out, Length: 13629, dtype: int64